In [1]:
import json
import datetime
import pytz
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error, r2_score
from statistics import mean
import statsmodels.api as sm
from sklearn.model_selection import KFold
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from statistics import mean
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.datasets import make_regression
from sklearn.preprocessing import OneHotEncoder
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import StandardScaler  
from sklearn.preprocessing import PolynomialFeatures
from sklearn.neighbors import KNeighborsRegressor
import datetime,time

In [2]:
path = "tweet_data/"

files = ["tweets_#gohawks.txt", "tweets_#gopatriots.txt", \
        "tweets_#nfl.txt", "tweets_#patriots.txt", \
        "tweets_#sb49.txt", "tweets_#superbowl.txt"]
topics = ["gohawks", "gopatriots", "nfl", "patriots", "sb49", "superbowl"]
test_files =["sample0_period1.txt","sample0_period2.txt","sample0_period3.txt", \
             "sample1_period1.txt","sample1_period2.txt","sample1_period3.txt","sample2_period1.txt",\
             "sample2_period2.txt","sample2_period3.txt"]

for i, fl in enumerate(files):
    print("files[" + str(i) + "] => " + fl)

files[0] => tweets_#gohawks.txt
files[1] => tweets_#gopatriots.txt
files[2] => tweets_#nfl.txt
files[3] => tweets_#patriots.txt
files[4] => tweets_#sb49.txt
files[5] => tweets_#superbowl.txt


In [3]:
def transfer_time(data_raw,time_type):
    
    pst_tz = pytz.timezone('America/Los_Angeles')
    
    # sort according to time
    pddata_raw = pd.DataFrame(data_raw,columns=['time','tweets','retweets','followers','mentioned',\
                                                'media','active','author','favourites_count','title'])
    pddata_raw = pddata_raw.sort_values(by = 'time')
    pddata_raw = pddata_raw.reset_index(drop=True)               

    # calculate hour index and minute index from time
    if time_type == 'hour':
        hour_accu = []
        for index, row in pddata_raw.iterrows():  
            p = datetime.datetime.fromtimestamp(row['time'], pst_tz)  
            hour_accu.append(((p.month-1)*31+p.day-14)*24+p.hour)                             
        pddata_raw['time'] = hour_accu
    elif time_type == 'minute':
        minu_accu = []
        for index, row in pddata_raw.iterrows():  
            p = datetime.datetime.fromtimestamp(row['time'], pst_tz)                    
            minu_accu.append((((p.month-1)*31+p.day-14)*24 + (p.hour-0))*12 + p.minute//5)             
        pddata_raw['time'] = minu_accu    
    else:
        print("Invalid time type")
        
    return pddata_raw

In [4]:
def generate_df(pddata_raw):
    """
    Create a new dataframe with desired form
    """
    df = pd.DataFrame([],columns=['time unit','tweets','retweets','followers sum','followers max',\
                                  'mentioned','media','active','author','favourites_count','title'])
    
    col = pddata_raw.columns.get_loc('time')
    df['time unit'] = range(int(pddata_raw.iloc[len(pddata_raw.index)-1,col] - pddata_raw.iloc[0,col]+1))

    df['tweets'] = pddata_raw.groupby("time")['tweets'].sum()
    df['retweets'] = pddata_raw.groupby("time")['retweets'].sum()
    df['followers sum'] = pddata_raw.groupby("time")['followers'].sum()
    df['followers max'] = pddata_raw.groupby("time")["followers"].max()
    df['mentioned'] = pddata_raw.groupby("time")['mentioned'].sum()
    df['media'] = pddata_raw.groupby("time")['media'].sum()
    df['active'] = pddata_raw.groupby("time")['active'].mean()  
    df['author'] = pddata_raw.groupby("time")['author'].nunique() # count number of not-repeating authors    
    df['favourites_count'] = pddata_raw.groupby("time")['favourites_count'].sum()
    df['title'] = pddata_raw.groupby("time")['title'].mean()
            
    # reset index of df
    df = df.fillna(0).reset_index(drop=True)
    
    return df

In [11]:
def parse_dataset(files):
    """
    Prase x conponent of the dataset into pandas DataFrame including columns of:
    tweets, retweets, total_followers, max_followers, mentioned, media, active, author, favourites_count, title
    with lines of hours,
    where mentioned: number of @ in tweets per hour
          media: number of url in tweets per hour
          active: a measure of active state of author
          author: number of unique authors post tweet per hour
          favourites_count: the total number of likes by this user
          title: length of this tweet's title
    Prase y of dataset as number of tweets in the next hour.
    """
    start_time = time.mktime(time.strptime("2015-02-01 08:00:00",'%Y-%m-%d %H:%M:%S'))
    end_time = time.mktime(time.strptime("2015-02-01 20:00:00",'%Y-%m-%d %H:%M:%S'))   

    start_hour_idx = ((2-1)*31+1-14)*24+8
    end_hour_idx = ((2-1)*31+1-14)*24+20
    start_minute_idx = (((2-1)*31+1-14)*24 + (8-0))*12 + 0//5    
        
    # extract raw features
    data_raw = [[],[],[]]
    for file in files:
        for line in open(path + file, 'r', encoding="utf-8") :
            row_tmp = []
            a = json.loads(line)
            citation_date = a['citation_date']
            tweet = 1
            retweet = a['metrics']['citations']['total']
            foll = a['author']['followers']             
            ment = len(a['tweet']['entities']['user_mentions'])        
            medi = len(a['tweet']['extended_entities']['media']) if 'extended_entities' in a['tweet'] else 0
            hist_tw = a['tweet']['user']["statuses_count"]
            hist_yr = a['tweet']['user']['created_at'][-4:]
            acti = hist_tw/(2015-float(hist_yr)+1) 
            auth = a['author']['name']
            favo = a['tweet']['user']['favourites_count']
            titl = len(a['title'])

            # append to row_tmp
            row_tmp.append(citation_date)        
            row_tmp.append(tweet)        
            row_tmp.append(retweet)
            row_tmp.append(foll)    
            row_tmp.append(ment) 
            row_tmp.append(medi) 
            row_tmp.append(acti)  
            row_tmp.append(auth)
            row_tmp.append(favo)
            row_tmp.append(titl)

            # assign to 3 periods
            if citation_date < start_time:
                data_raw[0].append(row_tmp)
            elif citation_date < end_time:
                data_raw[1].append(row_tmp)
            else:
                data_raw[2].append(row_tmp)            

    # generate raw pandas dataframe
    pddata_raw_1 = transfer_time(data_raw[0],'hour')
    pddata_raw_2 = transfer_time(data_raw[1],'minute')
    pddata_raw_2['time'] = pddata_raw_2['time'] - start_minute_idx
    pddata_raw_3 = transfer_time(data_raw[2],'hour')
    pddata_raw_3['time'] = pddata_raw_3['time'] - end_hour_idx - 1    
            
    # generate df and df_y for each time slot
    df_1 = generate_df(pddata_raw_1)  
    df_y_1 = df_1.iloc[1:,1].reset_index(drop=True)
    df_1 = df_1[:len(df_y_1)]
    
    df_2 = generate_df(pddata_raw_2)
    df_y_2 = df_2.iloc[1:,1].reset_index(drop=True)
    df_2 = df_2[:len(df_y_2)]
   
    df_3 = generate_df(pddata_raw_3)
    df_y_3 = df_3.iloc[1:,1].reset_index(drop=True)
    df_3 = df_3[:len(df_y_3)]
    
    return (df_1.iloc[:,1:],df_y_1), (df_2.iloc[:,1:],df_y_2), (df_3.iloc[:,1:],df_y_3)

In [6]:
def plot_recipe(df_y, pred_y):
    """
    This function plots fitted values vs true values
    """
    plt.figure()
    area = np.pi * (4)**2/4
    plt.scatter(df_y, pred_y, s = area)
    plt.plot([df_y.min(), df_y.max()], [df_y.min(), df_y.max()], 'k--', lw = 1)
    plt.xlabel('true values')
    plt.ylabel('fitted values')
    plt.show()

In [7]:

def ols_regression(df, df_y):
    X2 = sm.add_constant(df)
    y = df_y.as_matrix()
    lm = sm.OLS(y, X2).fit()
    print(lm.summary())
    print(list(df))

In [ ]:
(df,df_y),(df_2,df_y_2),(df_3,df_y_3) = parse_dataset(files)

In [22]:
param_grid={
'max_depth': [10, 20, 40, 60, 80, 100, 200, None],
'max_features': ['auto', 'sqrt'],
'min_samples_leaf': [1, 2, 4],
'min_samples_split': [2, 5, 10],
'n_estimators': [200, 400, 600, 800, 1000,
1200, 1400, 1600, 1800, 2000]
}
kf = KFold(n_splits=5,random_state=42,shuffle=True)

In [34]:
df_3.head()

,tweets,retweets,followers sum,followers max,mentioned,media,active,author,favourites_count,title
0,6492.0,52858.0,106280297.0,15997691.0,3689.0,1919.0,3527.647824,4568.0,15634832.0,101.818392
1,9037.0,48221.0,61182588.0,9675466.0,5516.0,2340.0,5242.472635,5732.0,17620137.0,103.982406
2,5018.0,16778.0,65565526.0,9675466.0,2313.0,1166.0,5628.756802,3186.0,8980699.0,103.580311
3,3292.0,14989.0,35632346.0,5025007.0,1449.0,779.0,6170.050290,2044.0,5958408.0,106.211422
4,2605.0,5965.0,16128911.0,3093728.0,1342.0,528.0,6052.328881,1531.0,4116194.0,106.714012


In [35]:
df_y_2.head()

0    1336
1    1231
2    1299
3    1230
4     321
Name: tweets, dtype: int64

# GradientBoostingRegressor 

In [24]:

avg_RMSE1_train=np.zeros((8,2,3,3,10))
avg_RMSE1_test=np.zeros((8,2,3,3,10))
for n1,max_depth in enumerate(param_grid['max_depth']):
    for n2,max_features in enumerate(param_grid['max_features']):
        for n3,min_samples_leaf in enumerate(param_grid['min_samples_leaf']):
            for n4,min_samples_split in enumerate(param_grid['min_samples_split']):
                for n5,n_estimators in enumerate(param_grid['n_estimators']):
                    MSE_train=[]
                    MSE_test=[]
                    oob_error=[]
                    total_train=0
                    total_test=0
                    for train_index, test_index in kf.split(df):
                        X_train= df.iloc[train_index]
                        y_train= df_y.iloc[train_index]
                        X_test= df.iloc[test_index]
                        y_test= df_y.iloc[test_index]
                        reg = GradientBoostingRegressor(n_estimators=n_estimators,max_depth=max_depth,min_samples_leaf=min_samples_leaf,min_samples_split=min_samples_split, max_features=max_features,random_state=42)
                        reg.fit(X_train,y_train)
                        pred_train = reg.predict(X_train)
                        pred_test = reg.predict(X_test)
                        MSE_train.append(mean_squared_error(y_train, pred_train)*len(train_index))
                        MSE_test.append(mean_squared_error(y_test, pred_test)*len(test_index))
                        total_train=total_train+len(train_index)
                        total_test=total_test+len(test_index)

                    avg_RMSE1_test[n1,n2,n3,n4,n5]=np.sqrt(sum(MSE_test)/total_test)
                    avg_RMSE1_train[n1,n2,n3,n4,n5]=np.sqrt(sum(MSE_train)/total_train)
                    print("max_depth=",max_depth,"max_features=",max_features,"min_samples_leaf=",min_samples_leaf,"min_samples_split=",min_samples_split,"n_estimators=",n_estimators)
                    print("RMSE_train=",avg_RMSE1_train[n1,n2,n3,n4,n5],"RMSE_test=",avg_RMSE1_test[n1,n2,n3,n4,n5])
                    print("")

max_depth= 10 max_features= auto min_samples_leaf= 1 min_samples_split= 2 n_estimators= 200
RMSE_train= 0.008602084514251651 RMSE_test= 2352.392887664536

max_depth= 10 max_features= auto min_samples_leaf= 1 min_samples_split= 2 n_estimators= 400
RMSE_train= 0.00031506004610060243 RMSE_test= 2352.392916893676

max_depth= 10 max_features= auto min_samples_leaf= 1 min_samples_split= 2 n_estimators= 600
RMSE_train= 0.00031506004610037225 RMSE_test= 2352.392916893676

max_depth= 10 max_features= auto min_samples_leaf= 1 min_samples_split= 2 n_estimators= 800
RMSE_train= 0.000315060046100242 RMSE_test= 2352.392916893676

max_depth= 10 max_features= auto min_samples_leaf= 1 min_samples_split= 2 n_estimators= 1000
RMSE_train= 0.00031506004610038776 RMSE_test= 2352.392916893676

max_depth= 10 max_features= auto min_samples_leaf= 1 min_samples_split= 2 n_estimators= 1200
RMSE_train= 0.00031506004610038776 RMSE_test= 2352.392916893676

max_depth= 10 max_features= auto min_samples_leaf= 1 min_sam

max_depth= 10 max_features= auto min_samples_leaf= 2 min_samples_split= 10 n_estimators= 800
RMSE_train= 0.00031517220945081846 RMSE_test= 1855.0801009314198

max_depth= 10 max_features= auto min_samples_leaf= 2 min_samples_split= 10 n_estimators= 1000
RMSE_train= 0.000315172209450754 RMSE_test= 1855.0801009314198

max_depth= 10 max_features= auto min_samples_leaf= 2 min_samples_split= 10 n_estimators= 1200
RMSE_train= 0.00031517220945091767 RMSE_test= 1855.0801009314198

max_depth= 10 max_features= auto min_samples_leaf= 2 min_samples_split= 10 n_estimators= 1400
RMSE_train= 0.00031517220945091767 RMSE_test= 1855.0801009314198

max_depth= 10 max_features= auto min_samples_leaf= 2 min_samples_split= 10 n_estimators= 1600
RMSE_train= 0.00031517220945091767 RMSE_test= 1855.0801009314198

max_depth= 10 max_features= auto min_samples_leaf= 2 min_samples_split= 10 n_estimators= 1800
RMSE_train= 0.00031517220945091767 RMSE_test= 1855.0801009314198

max_depth= 10 max_features= auto min_sample

max_depth= 10 max_features= sqrt min_samples_leaf= 1 min_samples_split= 5 n_estimators= 1400
RMSE_train= 0.0003145300890185942 RMSE_test= 2102.0493607292374

max_depth= 10 max_features= sqrt min_samples_leaf= 1 min_samples_split= 5 n_estimators= 1600
RMSE_train= 0.0003145300890185942 RMSE_test= 2102.0493607292374

max_depth= 10 max_features= sqrt min_samples_leaf= 1 min_samples_split= 5 n_estimators= 1800
RMSE_train= 0.0003145300890185942 RMSE_test= 2102.0493607292374

max_depth= 10 max_features= sqrt min_samples_leaf= 1 min_samples_split= 5 n_estimators= 2000
RMSE_train= 0.0003145300890185942 RMSE_test= 2102.0493607292374

max_depth= 10 max_features= sqrt min_samples_leaf= 1 min_samples_split= 10 n_estimators= 200
RMSE_train= 3.393879771756152 RMSE_test= 1922.4288031872832

max_depth= 10 max_features= sqrt min_samples_leaf= 1 min_samples_split= 10 n_estimators= 400
RMSE_train= 0.031919019805234865 RMSE_test= 1922.4655650835036

max_depth= 10 max_features= sqrt min_samples_leaf= 1 min_

max_depth= 10 max_features= sqrt min_samples_leaf= 4 min_samples_split= 2 n_estimators= 2000
RMSE_train= 0.00033575902039440343 RMSE_test= 2080.3735192738304

max_depth= 10 max_features= sqrt min_samples_leaf= 4 min_samples_split= 5 n_estimators= 200
RMSE_train= 142.6357810313224 RMSE_test= 2080.577449541427

max_depth= 10 max_features= sqrt min_samples_leaf= 4 min_samples_split= 5 n_estimators= 400
RMSE_train= 23.311849335962982 RMSE_test= 2080.6625553650206

max_depth= 10 max_features= sqrt min_samples_leaf= 4 min_samples_split= 5 n_estimators= 600
RMSE_train= 4.597276356229266 RMSE_test= 2080.603996016076

max_depth= 10 max_features= sqrt min_samples_leaf= 4 min_samples_split= 5 n_estimators= 800
RMSE_train= 0.8431258238098341 RMSE_test= 2080.3860440241892

max_depth= 10 max_features= sqrt min_samples_leaf= 4 min_samples_split= 5 n_estimators= 1000
RMSE_train= 0.17394146682441544 RMSE_test= 2080.3754996491166

max_depth= 10 max_features= sqrt min_samples_leaf= 4 min_samples_split= 5

max_depth= 20 max_features= auto min_samples_leaf= 2 min_samples_split= 2 n_estimators= 600
RMSE_train= 0.00031364364688611906 RMSE_test= 2116.432389002909

max_depth= 20 max_features= auto min_samples_leaf= 2 min_samples_split= 2 n_estimators= 800
RMSE_train= 0.00031364364688611906 RMSE_test= 2116.432389002909

max_depth= 20 max_features= auto min_samples_leaf= 2 min_samples_split= 2 n_estimators= 1000
RMSE_train= 0.00031364364688611906 RMSE_test= 2116.432389002909

max_depth= 20 max_features= auto min_samples_leaf= 2 min_samples_split= 2 n_estimators= 1200
RMSE_train= 0.00031364364688611906 RMSE_test= 2116.432389002909

max_depth= 20 max_features= auto min_samples_leaf= 2 min_samples_split= 2 n_estimators= 1400
RMSE_train= 0.00031364364688611906 RMSE_test= 2116.432389002909

max_depth= 20 max_features= auto min_samples_leaf= 2 min_samples_split= 2 n_estimators= 1600
RMSE_train= 0.00031364364688611906 RMSE_test= 2116.432389002909

max_depth= 20 max_features= auto min_samples_leaf= 2 m

...

max_depth= 200 max_features= sqrt min_samples_leaf= 2 min_samples_split= 5 n_estimators= 400
RMSE_train= 0.0029479326742370294 RMSE_test= 2059.458389295433

max_depth= 200 max_features= sqrt min_samples_leaf= 2 min_samples_split= 5 n_estimators= 600
RMSE_train= 0.0003145255707177171 RMSE_test= 2059.458745501209

max_depth= 200 max_features= sqrt min_samples_leaf= 2 min_samples_split= 5 n_estimators= 800
RMSE_train= 0.0003145255707177057 RMSE_test= 2059.458745501209

max_depth= 200 max_features= sqrt min_samples_leaf= 2 min_samples_split= 5 n_estimators= 1000
RMSE_train= 0.0003145255707183189 RMSE_test= 2059.458745501209

max_depth= 200 max_features= sqrt min_samples_leaf= 2 min_samples_split= 5 n_estimators= 1200
RMSE_train= 0.00031452557071895437 RMSE_test= 2059.458745501209

max_depth= 200 max_features= sqrt min_samples_leaf= 2 min_samples_split= 5 n_estimators= 1400
RMSE_train= 0.0003145255707195899 RMSE_test= 2059.458745501209

max_depth= 200 max_features= sqrt min_samples_leaf= 2 

max_depth= None max_features= auto min_samples_leaf= 1 min_samples_split= 2 n_estimators= 1000
RMSE_train= 0.00031542981931996836 RMSE_test= 2373.021985014264

max_depth= None max_features= auto min_samples_leaf= 1 min_samples_split= 2 n_estimators= 1200
RMSE_train= 0.00031542981931996836 RMSE_test= 2373.021985014264

max_depth= None max_features= auto min_samples_leaf= 1 min_samples_split= 2 n_estimators= 1400
RMSE_train= 0.00031542981931996836 RMSE_test= 2373.021985014264

max_depth= None max_features= auto min_samples_leaf= 1 min_samples_split= 2 n_estimators= 1600
RMSE_train= 0.00031542981931996836 RMSE_test= 2373.021985014264

max_depth= None max_features= auto min_samples_leaf= 1 min_samples_split= 2 n_estimators= 1800
RMSE_train= 0.00031542981931996836 RMSE_test= 2373.021985014264

max_depth= None max_features= auto min_samples_leaf= 1 min_samples_split= 2 n_estimators= 2000
RMSE_train= 0.00031542981931996836 RMSE_test= 2373.021985014264

max_depth= None max_features= auto min_s

max_depth= None max_features= auto min_samples_leaf= 2 min_samples_split= 10 n_estimators= 1400
RMSE_train= 0.000315269923649938 RMSE_test= 1831.6899786926142

max_depth= None max_features= auto min_samples_leaf= 2 min_samples_split= 10 n_estimators= 1600
RMSE_train= 0.000315269923649938 RMSE_test= 1831.6899786926142

max_depth= None max_features= auto min_samples_leaf= 2 min_samples_split= 10 n_estimators= 1800
RMSE_train= 0.000315269923649938 RMSE_test= 1831.6899786926142

max_depth= None max_features= auto min_samples_leaf= 2 min_samples_split= 10 n_estimators= 2000
RMSE_train= 0.000315269923649938 RMSE_test= 1831.6899786926142

max_depth= None max_features= auto min_samples_leaf= 4 min_samples_split= 2 n_estimators= 200
RMSE_train= 108.05073626069012 RMSE_test= 1963.527924030716

max_depth= None max_features= auto min_samples_leaf= 4 min_samples_split= 2 n_estimators= 400
RMSE_train= 11.917651273792996 RMSE_test= 1963.5814473064775

max_depth= None max_features= auto min_samples_le

max_depth= None max_features= sqrt min_samples_leaf= 1 min_samples_split= 5 n_estimators= 1800
RMSE_train= 0.0003150267500741105 RMSE_test= 2090.4141635868596

max_depth= None max_features= sqrt min_samples_leaf= 1 min_samples_split= 5 n_estimators= 2000
RMSE_train= 0.0003150267500741105 RMSE_test= 2090.4141635868596

max_depth= None max_features= sqrt min_samples_leaf= 1 min_samples_split= 10 n_estimators= 200
RMSE_train= 0.08350755933142631 RMSE_test= 2023.9915614737852

max_depth= None max_features= sqrt min_samples_leaf= 1 min_samples_split= 10 n_estimators= 400
RMSE_train= 0.00031450361988731264 RMSE_test= 2023.9927077324703

max_depth= None max_features= sqrt min_samples_leaf= 1 min_samples_split= 10 n_estimators= 600
RMSE_train= 0.00031450361988896573 RMSE_test= 2023.9927077324703

max_depth= None max_features= sqrt min_samples_leaf= 1 min_samples_split= 10 n_estimators= 800
RMSE_train= 0.000314503619889486 RMSE_test= 2023.9927077324703

max_depth= None max_features= sqrt min_sa

max_depth= None max_features= sqrt min_samples_leaf= 4 min_samples_split= 5 n_estimators= 200
RMSE_train= 138.15505079906745 RMSE_test= 2013.6351827920512

max_depth= None max_features= sqrt min_samples_leaf= 4 min_samples_split= 5 n_estimators= 400
RMSE_train= 22.344313701376905 RMSE_test= 2015.0622731825665

max_depth= None max_features= sqrt min_samples_leaf= 4 min_samples_split= 5 n_estimators= 600
RMSE_train= 4.578068480435749 RMSE_test= 2015.1631106412528

max_depth= None max_features= sqrt min_samples_leaf= 4 min_samples_split= 5 n_estimators= 800
RMSE_train= 0.8992387712442464 RMSE_test= 2015.0680271747678

max_depth= None max_features= sqrt min_samples_leaf= 4 min_samples_split= 5 n_estimators= 1000
RMSE_train= 0.18398457079126193 RMSE_test= 2015.0748635409375

max_depth= None max_features= sqrt min_samples_leaf= 4 min_samples_split= 5 n_estimators= 1200
RMSE_train= 0.040143464627964615 RMSE_test= 2015.067436241985

max_depth= None max_features= sqrt min_samples_leaf= 4 min_sa

In [25]:
print("min RMSE in testset=",np.min(avg_RMSE1_test))
print("parameters:")
n1,n2,n3,n4,n5= np.unravel_index(np.argmin(avg_RMSE1_test), avg_RMSE1_test.shape)
print("max_depth=",param_grid["max_depth"][n1],"max_features=",param_grid["max_features"][n2],"min_samples_leaf=",param_grid["min_samples_leaf"][n3],"min_samples_split=",param_grid["min_samples_split"][n4],"n_estimators=",param_grid["n_estimators"][n5])
print("")

min RMSE in testset= 1827.2817347419784
parameters:
max_depth= 20 max_features= auto min_samples_leaf= 2 min_samples_split= 10 n_estimators= 200



In [27]:
reg = GradientBoostingRegressor(n_estimators=param_grid["n_estimators"][n5],max_depth=param_grid["max_depth"][n1],min_samples_leaf=param_grid["min_samples_leaf"][n3],min_samples_split=param_grid["min_samples_split"][n4], max_features=param_grid["max_features"][n2],random_state=42)
reg.fit(df,df_y)
pred = reg.predict(df)
print("MSE=",mean_squared_error(df_y, pred))

MSE= 5.73599539627572


In [28]:

avg_RMSE2_train=np.zeros((8,2,3,3,10))
avg_RMSE2_test=np.zeros((8,2,3,3,10))
for n1,max_depth in enumerate(param_grid['max_depth']):
    for n2,max_features in enumerate(param_grid['max_features']):
        for n3,min_samples_leaf in enumerate(param_grid['min_samples_leaf']):
            for n4,min_samples_split in enumerate(param_grid['min_samples_split']):
                for n5,n_estimators in enumerate(param_grid['n_estimators']):
                    MSE_train=[]
                    MSE_test=[]
                    oob_error=[]
                    total_train=0
                    total_test=0
                    for train_index, test_index in kf.split(df_2):
                        X_train= df_2.iloc[train_index]
                        y_train= df_y_2.iloc[train_index]
                        X_test= df_2.iloc[test_index]
                        y_test= df_y_2.iloc[test_index]
                        reg = GradientBoostingRegressor(n_estimators=n_estimators,max_depth=max_depth,min_samples_leaf=min_samples_leaf,min_samples_split=min_samples_split, max_features=max_features,random_state=42)
                        reg.fit(X_train,y_train)
                        pred_train = reg.predict(X_train)
                        pred_test = reg.predict(X_test)
                        MSE_train.append(mean_squared_error(y_train, pred_train)*len(train_index))
                        MSE_test.append(mean_squared_error(y_test, pred_test)*len(test_index))
                        total_train=total_train+len(train_index)
                        total_test=total_test+len(test_index)

                    avg_RMSE2_test[n1,n2,n3,n4,n5]=np.sqrt(sum(MSE_test)/total_test)
                    avg_RMSE2_train[n1,n2,n3,n4,n5]=np.sqrt(sum(MSE_train)/total_train)
                    print("max_depth=",max_depth,"max_features=",max_features,"min_samples_leaf=",min_samples_leaf,"min_samples_split=",min_samples_split,"n_estimators=",n_estimators)
                    print("RMSE_train=",avg_RMSE2_train[n1,n2,n3,n4,n5],"RMSE_test=",avg_RMSE2_test[n1,n2,n3,n4,n5])
                    print("")

max_depth= 10 max_features= auto min_samples_leaf= 1 min_samples_split= 2 n_estimators= 200
RMSE_train= 0.0003138022494075089 RMSE_test= 5831.117077398146

max_depth= 10 max_features= auto min_samples_leaf= 1 min_samples_split= 2 n_estimators= 400
RMSE_train= 0.0003138022494519666 RMSE_test= 5831.117077398146

max_depth= 10 max_features= auto min_samples_leaf= 1 min_samples_split= 2 n_estimators= 600
RMSE_train= 0.00031380224945372484 RMSE_test= 5831.117077398146

max_depth= 10 max_features= auto min_samples_leaf= 1 min_samples_split= 2 n_estimators= 800
RMSE_train= 0.00031380224945372484 RMSE_test= 5831.117077398146

max_depth= 10 max_features= auto min_samples_leaf= 1 min_samples_split= 2 n_estimators= 1000
RMSE_train= 0.00031380224945372484 RMSE_test= 5831.117077398146

max_depth= 10 max_features= auto min_samples_leaf= 1 min_samples_split= 2 n_estimators= 1200
RMSE_train= 0.00031380224945372484 RMSE_test= 5831.117077398146

max_depth= 10 max_features= auto min_samples_leaf= 1 min_s

max_depth= 10 max_features= auto min_samples_leaf= 2 min_samples_split= 10 n_estimators= 800
RMSE_train= 0.00031506839215777793 RMSE_test= 4999.429451473608

max_depth= 10 max_features= auto min_samples_leaf= 2 min_samples_split= 10 n_estimators= 1000
RMSE_train= 0.0003150683921500537 RMSE_test= 4999.429451473608

max_depth= 10 max_features= auto min_samples_leaf= 2 min_samples_split= 10 n_estimators= 1200
RMSE_train= 0.0003150683921500537 RMSE_test= 4999.429451473608

max_depth= 10 max_features= auto min_samples_leaf= 2 min_samples_split= 10 n_estimators= 1400
RMSE_train= 0.0003150683921500537 RMSE_test= 4999.429451473608

max_depth= 10 max_features= auto min_samples_leaf= 2 min_samples_split= 10 n_estimators= 1600
RMSE_train= 0.0003150683921500537 RMSE_test= 4999.429451473608

max_depth= 10 max_features= auto min_samples_leaf= 2 min_samples_split= 10 n_estimators= 1800
RMSE_train= 0.0003150683921500537 RMSE_test= 4999.429451473608

max_depth= 10 max_features= auto min_samples_leaf= 2

max_depth= 10 max_features= sqrt min_samples_leaf= 1 min_samples_split= 5 n_estimators= 1400
RMSE_train= 0.0003135800549489076 RMSE_test= 4916.674392948595

max_depth= 10 max_features= sqrt min_samples_leaf= 1 min_samples_split= 5 n_estimators= 1600
RMSE_train= 0.0003135800549489076 RMSE_test= 4916.674392948595

max_depth= 10 max_features= sqrt min_samples_leaf= 1 min_samples_split= 5 n_estimators= 1800
RMSE_train= 0.0003135800549489076 RMSE_test= 4916.674392948595

max_depth= 10 max_features= sqrt min_samples_leaf= 1 min_samples_split= 5 n_estimators= 2000
RMSE_train= 0.0003135800549489076 RMSE_test= 4916.674392948595

max_depth= 10 max_features= sqrt min_samples_leaf= 1 min_samples_split= 10 n_estimators= 200
RMSE_train= 1.20747405669576 RMSE_test= 4892.805618307515

max_depth= 10 max_features= sqrt min_samples_leaf= 1 min_samples_split= 10 n_estimators= 400
RMSE_train= 0.0021488202757294437 RMSE_test= 4892.872252395578

max_depth= 10 max_features= sqrt min_samples_leaf= 1 min_sample

max_depth= 10 max_features= sqrt min_samples_leaf= 4 min_samples_split= 2 n_estimators= 2000
RMSE_train= 0.0003154902334091034 RMSE_test= 4405.162914347088

max_depth= 10 max_features= sqrt min_samples_leaf= 4 min_samples_split= 5 n_estimators= 200
RMSE_train= 56.19421099288044 RMSE_test= 4415.654260075423

max_depth= 10 max_features= sqrt min_samples_leaf= 4 min_samples_split= 5 n_estimators= 400
RMSE_train= 2.7069621239373585 RMSE_test= 4405.440118876645

max_depth= 10 max_features= sqrt min_samples_leaf= 4 min_samples_split= 5 n_estimators= 600
RMSE_train= 0.0957126808472182 RMSE_test= 4405.176895349282

max_depth= 10 max_features= sqrt min_samples_leaf= 4 min_samples_split= 5 n_estimators= 800
RMSE_train= 0.0036244442894597926 RMSE_test= 4405.163427104113

max_depth= 10 max_features= sqrt min_samples_leaf= 4 min_samples_split= 5 n_estimators= 1000
RMSE_train= 0.00034925320153898466 RMSE_test= 4405.162911961383

max_depth= 10 max_features= sqrt min_samples_leaf= 4 min_samples_split=

max_depth= 20 max_features= auto min_samples_leaf= 2 min_samples_split= 2 n_estimators= 600
RMSE_train= 0.00031367162998903426 RMSE_test= 5468.812973806835

max_depth= 20 max_features= auto min_samples_leaf= 2 min_samples_split= 2 n_estimators= 800
RMSE_train= 0.0003136716299913153 RMSE_test= 5468.812973806835

max_depth= 20 max_features= auto min_samples_leaf= 2 min_samples_split= 2 n_estimators= 1000
RMSE_train= 0.0003136716299913153 RMSE_test= 5468.812973806835

max_depth= 20 max_features= auto min_samples_leaf= 2 min_samples_split= 2 n_estimators= 1200
RMSE_train= 0.0003136716299913153 RMSE_test= 5468.812973806835

max_depth= 20 max_features= auto min_samples_leaf= 2 min_samples_split= 2 n_estimators= 1400
RMSE_train= 0.0003136716299913153 RMSE_test= 5468.812973806835

max_depth= 20 max_features= auto min_samples_leaf= 2 min_samples_split= 2 n_estimators= 1600
RMSE_train= 0.0003136716299913153 RMSE_test= 5468.812973806835

max_depth= 20 max_features= auto min_samples_leaf= 2 min_sa

...

max_depth= 200 max_features= sqrt min_samples_leaf= 2 min_samples_split= 5 n_estimators= 1400
RMSE_train= 0.0003155407479771634 RMSE_test= 4825.431024774192

max_depth= 200 max_features= sqrt min_samples_leaf= 2 min_samples_split= 5 n_estimators= 1600
RMSE_train= 0.0003155407479771634 RMSE_test= 4825.431024774192

max_depth= 200 max_features= sqrt min_samples_leaf= 2 min_samples_split= 5 n_estimators= 1800
RMSE_train= 0.0003155407479771634 RMSE_test= 4825.431024774192

max_depth= 200 max_features= sqrt min_samples_leaf= 2 min_samples_split= 5 n_estimators= 2000
RMSE_train= 0.0003155407479771634 RMSE_test= 4825.431024774192

max_depth= 200 max_features= sqrt min_samples_leaf= 2 min_samples_split= 10 n_estimators= 200
RMSE_train= 2.622288865753821 RMSE_test= 4587.99099350324

max_depth= 200 max_features= sqrt min_samples_leaf= 2 min_samples_split= 10 n_estimators= 400
RMSE_train= 0.004006511965084205 RMSE_test= 4588.121537921316

max_depth= 200 max_features= sqrt min_samples_leaf= 2 min_

max_depth= None max_features= auto min_samples_leaf= 1 min_samples_split= 2 n_estimators= 1800
RMSE_train= 0.0003128895962767885 RMSE_test= 5883.85552880857

max_depth= None max_features= auto min_samples_leaf= 1 min_samples_split= 2 n_estimators= 2000
RMSE_train= 0.0003128895962767885 RMSE_test= 5883.85552880857

max_depth= None max_features= auto min_samples_leaf= 1 min_samples_split= 5 n_estimators= 200
RMSE_train= 0.0009701053772086372 RMSE_test= 5438.331384636167

max_depth= None max_features= auto min_samples_leaf= 1 min_samples_split= 5 n_estimators= 400
RMSE_train= 0.00031548966623371876 RMSE_test= 5438.331383979046

max_depth= None max_features= auto min_samples_leaf= 1 min_samples_split= 5 n_estimators= 600
RMSE_train= 0.0003154896662254938 RMSE_test= 5438.331383979046

max_depth= None max_features= auto min_samples_leaf= 1 min_samples_split= 5 n_estimators= 800
RMSE_train= 0.0003154896662254938 RMSE_test= 5438.331383979046

max_depth= None max_features= auto min_samples_leaf

max_depth= None max_features= auto min_samples_leaf= 4 min_samples_split= 2 n_estimators= 200
RMSE_train= 50.93263430233771 RMSE_test= 4706.691695417675

max_depth= None max_features= auto min_samples_leaf= 4 min_samples_split= 2 n_estimators= 400
RMSE_train= 1.9669066910289743 RMSE_test= 4698.086874270756

max_depth= None max_features= auto min_samples_leaf= 4 min_samples_split= 2 n_estimators= 600
RMSE_train= 0.130603904267227 RMSE_test= 4698.174202975068

max_depth= None max_features= auto min_samples_leaf= 4 min_samples_split= 2 n_estimators= 800
RMSE_train= 0.010074136577810724 RMSE_test= 4698.172073641119

max_depth= None max_features= auto min_samples_leaf= 4 min_samples_split= 2 n_estimators= 1000
RMSE_train= 0.0009217272510292027 RMSE_test= 4698.171473288572

max_depth= None max_features= auto min_samples_leaf= 4 min_samples_split= 2 n_estimators= 1200
RMSE_train= 0.000314468748347336 RMSE_test= 4698.171332947795

max_depth= None max_features= auto min_samples_leaf= 4 min_samp

max_depth= None max_features= sqrt min_samples_leaf= 1 min_samples_split= 10 n_estimators= 600
RMSE_train= 0.00031396753794319423 RMSE_test= 4909.541793957797

max_depth= None max_features= sqrt min_samples_leaf= 1 min_samples_split= 10 n_estimators= 800
RMSE_train= 0.0003139675379321913 RMSE_test= 4909.541793957797

max_depth= None max_features= sqrt min_samples_leaf= 1 min_samples_split= 10 n_estimators= 1000
RMSE_train= 0.0003139675379321913 RMSE_test= 4909.541793957797

max_depth= None max_features= sqrt min_samples_leaf= 1 min_samples_split= 10 n_estimators= 1200
RMSE_train= 0.0003139675379321913 RMSE_test= 4909.541793957797

max_depth= None max_features= sqrt min_samples_leaf= 1 min_samples_split= 10 n_estimators= 1400
RMSE_train= 0.0003139675379321913 RMSE_test= 4909.541793957797

max_depth= None max_features= sqrt min_samples_leaf= 1 min_samples_split= 10 n_estimators= 1600
RMSE_train= 0.0003139675379321913 RMSE_test= 4909.541793957797

max_depth= None max_features= sqrt min_sa

max_depth= None max_features= sqrt min_samples_leaf= 4 min_samples_split= 5 n_estimators= 1000
RMSE_train= 0.00046949957855583834 RMSE_test= 4380.2580612784905

max_depth= None max_features= sqrt min_samples_leaf= 4 min_samples_split= 5 n_estimators= 1200
RMSE_train= 0.0003155982480991678 RMSE_test= 4380.258019506062

max_depth= None max_features= sqrt min_samples_leaf= 4 min_samples_split= 5 n_estimators= 1400
RMSE_train= 0.00031559824808292524 RMSE_test= 4380.258019506062

max_depth= None max_features= sqrt min_samples_leaf= 4 min_samples_split= 5 n_estimators= 1600
RMSE_train= 0.0003155982480761398 RMSE_test= 4380.258019506062

max_depth= None max_features= sqrt min_samples_leaf= 4 min_samples_split= 5 n_estimators= 1800
RMSE_train= 0.0003155982480761398 RMSE_test= 4380.258019506062

max_depth= None max_features= sqrt min_samples_leaf= 4 min_samples_split= 5 n_estimators= 2000
RMSE_train= 0.0003155982480761398 RMSE_test= 4380.258019506062

max_depth= None max_features= sqrt min_samp

In [29]:
print("min RMSE in testset=",np.min(avg_RMSE2_test))
print("parameters:")
n1,n2,n3,n4,n5= np.unravel_index(np.argmin(avg_RMSE2_test), avg_RMSE2_test.shape)
print("max_depth=",param_grid["max_depth"][n1],"max_features=",param_grid["max_features"][n2],"min_samples_leaf=",param_grid["min_samples_leaf"][n3],"min_samples_split=",param_grid["min_samples_split"][n4],"n_estimators=",param_grid["n_estimators"][n5])
print("")

min RMSE in testset= 4380.139096602064
parameters:
max_depth= 20 max_features= sqrt min_samples_leaf= 4 min_samples_split= 2 n_estimators= 1200



In [30]:
reg = GradientBoostingRegressor(n_estimators=param_grid["n_estimators"][n5],max_depth=param_grid["max_depth"][n1],min_samples_leaf=param_grid["min_samples_leaf"][n3],min_samples_split=param_grid["min_samples_split"][n4], max_features=param_grid["max_features"][n2],random_state=42)
reg.fit(df_2,df_y_2)
pred = reg.predict(df_2)
print("MSE=",mean_squared_error(df_y_2, pred))

MSE= 9.997359714340548e-08


In [36]:

avg_RMSE3_train=np.zeros((8,2,3,3,10))
avg_RMSE3_test=np.zeros((8,2,3,3,10))
for n1,max_depth in enumerate(param_grid['max_depth']):
    for n2,max_features in enumerate(param_grid['max_features']):
        for n3,min_samples_leaf in enumerate(param_grid['min_samples_leaf']):
            for n4,min_samples_split in enumerate(param_grid['min_samples_split']):
                for n5,n_estimators in enumerate(param_grid['n_estimators']):
                    MSE_train=[]
                    MSE_test=[]
                    oob_error=[]
                    total_train=0
                    total_test=0
                    for train_index, test_index in kf.split(df_3):
                        X_train= df_3.iloc[train_index]
                        y_train= df_y_3.iloc[train_index]
                        X_test= df_3.iloc[test_index]
                        y_test= df_y_3.iloc[test_index]
                        reg = GradientBoostingRegressor(n_estimators=n_estimators,max_depth=max_depth,min_samples_leaf=min_samples_leaf,min_samples_split=min_samples_split, max_features=max_features,random_state=42)
                        reg.fit(X_train,y_train)
                        pred_train = reg.predict(X_train)
                        pred_test = reg.predict(X_test)
                        MSE_train.append(mean_squared_error(y_train, pred_train)*len(train_index))
                        MSE_test.append(mean_squared_error(y_test, pred_test)*len(test_index))
                        total_train=total_train+len(train_index)
                        total_test=total_test+len(test_index)

                    avg_RMSE3_test[n1,n2,n3,n4,n5]=np.sqrt(sum(MSE_test)/total_test)
                    avg_RMSE3_train[n1,n2,n3,n4,n5]=np.sqrt(sum(MSE_train)/total_train)
                    print("max_depth=",max_depth,"max_features=",max_features,"min_samples_leaf=",min_samples_leaf,"min_samples_split=",min_samples_split,"n_estimators=",n_estimators)
                    print("RMSE_train=",avg_RMSE3_train[n1,n2,n3,n4,n5],"RMSE_test=",avg_RMSE3_test[n1,n2,n3,n4,n5])
                    print("")

max_depth= 10 max_features= auto min_samples_leaf= 1 min_samples_split= 2 n_estimators= 200
RMSE_train= 0.00031421488672659384 RMSE_test= 845.1080952436225

max_depth= 10 max_features= auto min_samples_leaf= 1 min_samples_split= 2 n_estimators= 400
RMSE_train= 0.00031421488673717034 RMSE_test= 845.1080952436225

max_depth= 10 max_features= auto min_samples_leaf= 1 min_samples_split= 2 n_estimators= 600
RMSE_train= 0.0003142148867392243 RMSE_test= 845.1080952436225

max_depth= 10 max_features= auto min_samples_leaf= 1 min_samples_split= 2 n_estimators= 800
RMSE_train= 0.0003142148867409118 RMSE_test= 845.1080952436225

max_depth= 10 max_features= auto min_samples_leaf= 1 min_samples_split= 2 n_estimators= 1000
RMSE_train= 0.00031421488674083695 RMSE_test= 845.1080952436225

max_depth= 10 max_features= auto min_samples_leaf= 1 min_samples_split= 2 n_estimators= 1200
RMSE_train= 0.00031421488674051283 RMSE_test= 845.1080952436225

max_depth= 10 max_features= auto min_samples_leaf= 1 min_s

max_depth= 10 max_features= auto min_samples_leaf= 2 min_samples_split= 10 n_estimators= 800
RMSE_train= 0.0003155752721331144 RMSE_test= 641.6006608936717

max_depth= 10 max_features= auto min_samples_leaf= 2 min_samples_split= 10 n_estimators= 1000
RMSE_train= 0.000315575272134212 RMSE_test= 641.6006608936717

max_depth= 10 max_features= auto min_samples_leaf= 2 min_samples_split= 10 n_estimators= 1200
RMSE_train= 0.0003155752721348269 RMSE_test= 641.6006608936717

max_depth= 10 max_features= auto min_samples_leaf= 2 min_samples_split= 10 n_estimators= 1400
RMSE_train= 0.0003155752721350243 RMSE_test= 641.6006608936717

max_depth= 10 max_features= auto min_samples_leaf= 2 min_samples_split= 10 n_estimators= 1600
RMSE_train= 0.0003155752721351881 RMSE_test= 641.6006608936717

max_depth= 10 max_features= auto min_samples_leaf= 2 min_samples_split= 10 n_estimators= 1800
RMSE_train= 0.0003155752721353239 RMSE_test= 641.6006608936717

max_depth= 10 max_features= auto min_samples_leaf= 2 m

max_depth= 10 max_features= sqrt min_samples_leaf= 1 min_samples_split= 5 n_estimators= 1400
RMSE_train= 0.0003154452262351648 RMSE_test= 695.5920545540802

max_depth= 10 max_features= sqrt min_samples_leaf= 1 min_samples_split= 5 n_estimators= 1600
RMSE_train= 0.0003154452262354094 RMSE_test= 695.5920545540802

max_depth= 10 max_features= sqrt min_samples_leaf= 1 min_samples_split= 5 n_estimators= 1800
RMSE_train= 0.0003154452262356123 RMSE_test= 695.5920545540802

max_depth= 10 max_features= sqrt min_samples_leaf= 1 min_samples_split= 5 n_estimators= 2000
RMSE_train= 0.00031544522623578057 RMSE_test= 695.5920545540802

max_depth= 10 max_features= sqrt min_samples_leaf= 1 min_samples_split= 10 n_estimators= 200
RMSE_train= 0.12343031054291309 RMSE_test= 670.9434436104322

max_depth= 10 max_features= sqrt min_samples_leaf= 1 min_samples_split= 10 n_estimators= 400
RMSE_train= 0.0003149289305329086 RMSE_test= 670.9452685534554

max_depth= 10 max_features= sqrt min_samples_leaf= 1 min_sa

max_depth= 10 max_features= sqrt min_samples_leaf= 4 min_samples_split= 2 n_estimators= 2000
RMSE_train= 0.00031511723023498983 RMSE_test= 557.9597833852035

max_depth= 10 max_features= sqrt min_samples_leaf= 4 min_samples_split= 5 n_estimators= 200
RMSE_train= 11.709639086298612 RMSE_test= 558.1353353606274

max_depth= 10 max_features= sqrt min_samples_leaf= 4 min_samples_split= 5 n_estimators= 400
RMSE_train= 1.2254847573521754 RMSE_test= 558.0175229537583

max_depth= 10 max_features= sqrt min_samples_leaf= 4 min_samples_split= 5 n_estimators= 600
RMSE_train= 0.17388374288105915 RMSE_test= 557.9619160777977

max_depth= 10 max_features= sqrt min_samples_leaf= 4 min_samples_split= 5 n_estimators= 800
RMSE_train= 0.02851143232068158 RMSE_test= 557.9560284470289

max_depth= 10 max_features= sqrt min_samples_leaf= 4 min_samples_split= 5 n_estimators= 1000
RMSE_train= 0.004569413198644397 RMSE_test= 557.9594344632378

max_depth= 10 max_features= sqrt min_samples_leaf= 4 min_samples_split= 

max_depth= 20 max_features= auto min_samples_leaf= 2 min_samples_split= 2 n_estimators= 600
RMSE_train= 0.0003138963946141704 RMSE_test= 724.9528436815621

max_depth= 20 max_features= auto min_samples_leaf= 2 min_samples_split= 2 n_estimators= 800
RMSE_train= 0.00031389639461110435 RMSE_test= 724.9528436815621

max_depth= 20 max_features= auto min_samples_leaf= 2 min_samples_split= 2 n_estimators= 1000
RMSE_train= 0.0003138963946089557 RMSE_test= 724.9528436815621

max_depth= 20 max_features= auto min_samples_leaf= 2 min_samples_split= 2 n_estimators= 1200
RMSE_train= 0.0003138963946071844 RMSE_test= 724.9528436815621

max_depth= 20 max_features= auto min_samples_leaf= 2 min_samples_split= 2 n_estimators= 1400
RMSE_train= 0.00031389639460583356 RMSE_test= 724.9528436815621

max_depth= 20 max_features= auto min_samples_leaf= 2 min_samples_split= 2 n_estimators= 1600
RMSE_train= 0.0003138963946047131 RMSE_test= 724.9528436815621

max_depth= 20 max_features= auto min_samples_leaf= 2 min_s

...

max_depth= 200 max_features= sqrt min_samples_leaf= 2 min_samples_split= 5 n_estimators= 1000
RMSE_train= 0.00031516556544536 RMSE_test= 677.2102302198547

max_depth= 200 max_features= sqrt min_samples_leaf= 2 min_samples_split= 5 n_estimators= 1200
RMSE_train= 0.00031516556544538096 RMSE_test= 677.2102302198547

max_depth= 200 max_features= sqrt min_samples_leaf= 2 min_samples_split= 5 n_estimators= 1400
RMSE_train= 0.00031516556544539836 RMSE_test= 677.2102302198547

max_depth= 200 max_features= sqrt min_samples_leaf= 2 min_samples_split= 5 n_estimators= 1600
RMSE_train= 0.0003151655654454128 RMSE_test= 677.2102302198547

max_depth= 200 max_features= sqrt min_samples_leaf= 2 min_samples_split= 5 n_estimators= 1800
RMSE_train= 0.00031516556544542476 RMSE_test= 677.2102302198547

max_depth= 200 max_features= sqrt min_samples_leaf= 2 min_samples_split= 5 n_estimators= 2000
RMSE_train= 0.0003151655654454347 RMSE_test= 677.2102302198547

max_depth= 200 max_features= sqrt min_samples_leaf=

max_depth= None max_features= auto min_samples_leaf= 1 min_samples_split= 2 n_estimators= 1600
RMSE_train= 0.0003135522604625882 RMSE_test= 832.0744253000655

max_depth= None max_features= auto min_samples_leaf= 1 min_samples_split= 2 n_estimators= 1800
RMSE_train= 0.00031355226046340343 RMSE_test= 832.0744253000655

max_depth= None max_features= auto min_samples_leaf= 1 min_samples_split= 2 n_estimators= 2000
RMSE_train= 0.00031355226046407965 RMSE_test= 832.0744253000655

max_depth= None max_features= auto min_samples_leaf= 1 min_samples_split= 5 n_estimators= 200
RMSE_train= 0.00031500000232238795 RMSE_test= 709.8605085920517

max_depth= None max_features= auto min_samples_leaf= 1 min_samples_split= 5 n_estimators= 400
RMSE_train= 0.000315000002326277 RMSE_test= 709.8605085920517

max_depth= None max_features= auto min_samples_leaf= 1 min_samples_split= 5 n_estimators= 600
RMSE_train= 0.00031500000232738764 RMSE_test= 709.8605085920517

max_depth= None max_features= auto min_samples

max_depth= None max_features= auto min_samples_leaf= 2 min_samples_split= 10 n_estimators= 2000
RMSE_train= 0.0003151564832966534 RMSE_test= 642.3030540932621

max_depth= None max_features= auto min_samples_leaf= 4 min_samples_split= 2 n_estimators= 200
RMSE_train= 11.026819662402243 RMSE_test= 606.7913142266123

max_depth= None max_features= auto min_samples_leaf= 4 min_samples_split= 2 n_estimators= 400
RMSE_train= 0.6698715826069228 RMSE_test= 608.9225955088494

max_depth= None max_features= auto min_samples_leaf= 4 min_samples_split= 2 n_estimators= 600
RMSE_train= 0.062495394880814764 RMSE_test= 608.9479385651791

max_depth= None max_features= auto min_samples_leaf= 4 min_samples_split= 2 n_estimators= 800
RMSE_train= 0.007451084736540967 RMSE_test= 608.9443556263817

max_depth= None max_features= auto min_samples_leaf= 4 min_samples_split= 2 n_estimators= 1000
RMSE_train= 0.0011241689684030442 RMSE_test= 608.9438697415533

max_depth= None max_features= auto min_samples_leaf= 4 mi

max_depth= None max_features= sqrt min_samples_leaf= 1 min_samples_split= 10 n_estimators= 400
RMSE_train= 0.00031422478781351217 RMSE_test= 674.323764120135

max_depth= None max_features= sqrt min_samples_leaf= 1 min_samples_split= 10 n_estimators= 600
RMSE_train= 0.0003142247878171886 RMSE_test= 674.323764120135

max_depth= None max_features= sqrt min_samples_leaf= 1 min_samples_split= 10 n_estimators= 800
RMSE_train= 0.00031422478781787435 RMSE_test= 674.323764120135

max_depth= None max_features= sqrt min_samples_leaf= 1 min_samples_split= 10 n_estimators= 1000
RMSE_train= 0.00031422478781843537 RMSE_test= 674.323764120135

max_depth= None max_features= sqrt min_samples_leaf= 1 min_samples_split= 10 n_estimators= 1200
RMSE_train= 0.0003142247878189007 RMSE_test= 674.323764120135

max_depth= None max_features= sqrt min_samples_leaf= 1 min_samples_split= 10 n_estimators= 1400
RMSE_train= 0.0003142247878192867 RMSE_test= 674.323764120135

max_depth= None max_features= sqrt min_samples

max_depth= None max_features= sqrt min_samples_leaf= 4 min_samples_split= 5 n_estimators= 800
RMSE_train= 0.0236038334314001 RMSE_test= 555.056768140396

max_depth= None max_features= sqrt min_samples_leaf= 4 min_samples_split= 5 n_estimators= 1000
RMSE_train= 0.0039016403096045157 RMSE_test= 555.0602454248329

max_depth= None max_features= sqrt min_samples_leaf= 4 min_samples_split= 5 n_estimators= 1200
RMSE_train= 0.001004646337678597 RMSE_test= 555.0606165496561

max_depth= None max_features= sqrt min_samples_leaf= 4 min_samples_split= 5 n_estimators= 1400
RMSE_train= 0.00040894892542635454 RMSE_test= 555.0606535368228

max_depth= None max_features= sqrt min_samples_leaf= 4 min_samples_split= 5 n_estimators= 1600
RMSE_train= 0.0003292036884015547 RMSE_test= 555.0606434596267

max_depth= None max_features= sqrt min_samples_leaf= 4 min_samples_split= 5 n_estimators= 1800
RMSE_train= 0.00031561749782754993 RMSE_test= 555.0606396469957

max_depth= None max_features= sqrt min_samples_lea

In [37]:
print("min RMSE in testset=",np.min(avg_RMSE3_test))
print("parameters:")
n1,n2,n3,n4,n5= np.unravel_index(np.argmin(avg_RMSE3_test), avg_RMSE3_test.shape)
print("max_depth=",param_grid["max_depth"][n1],"max_features=",param_grid["max_features"][n2],"min_samples_leaf=",param_grid["min_samples_leaf"][n3],"min_samples_split=",param_grid["min_samples_split"][n4],"n_estimators=",param_grid["n_estimators"][n5])
print("")

min RMSE in testset= 554.5984140323671
parameters:
max_depth= 20 max_features= sqrt min_samples_leaf= 4 min_samples_split= 2 n_estimators= 200



In [38]:
reg = GradientBoostingRegressor(n_estimators=param_grid["n_estimators"][n5],max_depth=param_grid["max_depth"][n1],min_samples_leaf=param_grid["min_samples_leaf"][n3],min_samples_split=param_grid["min_samples_split"][n4], max_features=param_grid["max_features"][n2],random_state=42)
reg.fit(df_3,df_y_3)
pred = reg.predict(df_3)
print("MSE=",mean_squared_error(df_y_3, pred))

MSE= 125.4351430453349
